# Optimizers:

---

### 1.   SGD [keras.optimizers.SGD(lr=0.01, momentum=0.0, decay=0.0, nesterov=False)]
### 2.   RMSProp [keras.optimizers.RMSprop(lr=0.001, rho=0.9, epsilon=None, decay=0.0***)]
### 3. AdaGrad [keras.optimizers.Adagrad(lr=0.01, epsilon=None, decay=0.0)]
### 4. Adam [keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)]



# First import required libraries and Keras layers

In [1]:
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K

Using TensorFlow backend.


# Define hyperparameters:
---


## Batchsize, number of epochs to train, number of classes & input image dimensions

In [0]:
batch_size = 128
num_classes = 10
epochs = 4
number_of_training_samples = 1000
number_of_test_samples = 500

img_rows, img_cols = 28, 28

# Load MNIST dataset & reshape the images

In [3]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
input_shape = (img_rows, img_cols, 1)

11493376/11490434 [==============================] - 0s 0us/step


# Normalize image pixels & check for dataset size

In [4]:
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print('Train samples: {} --- Test samples: {}'.format(x_train.shape[0], x_test.shape[0]))
print('Train labels: {} --- Test lables: {}'.format(y_train.shape[0], y_test.shape[0]))


x_train shape: (60000, 28, 28, 1)
Train samples: 60000 --- Test samples: 10000
Train labels: 60000 --- Test lables: 10000


# Decrease the dataset size so that we can train the model with several different optimizers

In [5]:
x_train = x_train[:number_of_training_samples, :, :, :]
y_train = y_train[:number_of_training_samples]
x_test = x_test[:number_of_test_samples, :, :, :]
y_test = y_test[:number_of_test_samples]

print('x_train shape:', x_train.shape)
print('Train samples: {} --- Test samples: {}'.format(x_train.shape[0], x_test.shape[0]))
print('Train labels: {} --- Test lables: {}'.format(y_train.shape[0], y_test.shape[0]))

x_train shape: (1000, 28, 28, 1)
Train samples: 1000 --- Test samples: 500
Train labels: 1000 --- Test lables: 500


# Convert class vectors to binary class matrices

In [0]:
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

# Define loss function, optimization algorithm & performance metrics

In [0]:
loss_ = keras.losses.categorical_crossentropy
"""
keras.optimizers.SGD(lr=0.01, momentum=0.0, decay=0.0, nesterov=False)
keras.optimizers.SGD(lr=0.01, momentum=0.9, decay=1e-6, nesterov=False)
keras.optimizers.SGD(lr=0.01, momentum=0.9, decay=1e-6, nesterov=True)



keras.optimizers.RMSprop(lr=0.001, rho=0.9, epsilon=None, decay=0.0)
keras.optimizers.Adagrad(lr=0.01, epsilon=None, decay=0.0)
keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
keras.optimizers.Adadelta()
"""
optimizer_ = keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)

metrics_ = ['accuracy']

# Build your CNN network 

In [15]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss=loss_,
              optimizer=optimizer_,
              metrics=metrics_)
model.summary()


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_5 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 24, 24, 64)        18496     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 12, 12, 64)        0         
_________________________________________________________________
dropout_5 (Dropout)          (None, 12, 12, 64)        0         
_________________________________________________________________
flatten_3 (Flatten)          (None, 9216)              0         
_________________________________________________________________
dense_5 (Dense)              (None, 32)                294944    
_________________________________________________________________
dropout_6 (Dropout)          (None, 32)                0         
__________

# Train the CNN with the pre-defined hyperparameters

In [16]:
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test))

Train on 1000 samples, validate on 500 samples
Epoch 1/4
1000/1000 [==============================] - 3s 3ms/step - loss: 2.1473 - acc: 0.2620 - val_loss: 1.8076 - val_acc: 0.6060
Epoch 2/4
1000/1000 [==============================] - 3s 3ms/step - loss: 1.5689 - acc: 0.4610 - val_loss: 1.1365 - val_acc: 0.6800
Epoch 3/4
1000/1000 [==============================] - 3s 3ms/step - loss: 1.1620 - acc: 0.6020 - val_loss: 0.8432 - val_acc: 0.7600
Epoch 4/4
1000/1000 [==============================] - 3s 3ms/step - loss: 0.9276 - acc: 0.6840 - val_loss: 0.6683 - val_acc: 0.8080


# Chech for performance metrics

---

## Test Loss & Accuracy

In [10]:
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 2.270305294036865
Test accuracy: 0.25600000047683713
